In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

## Cleaning MICROBIOLOGYEVENTS.csv

### Keeping only the required columns: SUBJECT_ID, HADM_ID, ORG_ITEMID, AB_ITEMID

In [2]:
MICROBIOLOGY_EVENTS_PATH = "./data/MICROBIOLOGYEVENTS.csv"

microbiology_events_df = pd.read_csv(MICROBIOLOGY_EVENTS_PATH)

display(microbiology_events_df.head())
display(microbiology_events_df.shape)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(631726, 16)

In [3]:
# Retaining only the required columns for processing data
cleaned_microbiology_events_df = microbiology_events_df.filter(["SUBJECT_ID", "HADM_ID", "ORG_ITEMID", "AB_ITEMID"])

display(cleaned_microbiology_events_df.head())
display(cleaned_microbiology_events_df.shape)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID
0,96,170324,80026.0,NaN
1,96,170324,NaN,NaN
2,96,170324,NaN,NaN
3,96,170324,NaN,NaN
4,96,170324,NaN,NaN


(631726, 4)

In [4]:
# removing rows with any columns containing NaN values
cleaned_microbiology_events_df = cleaned_microbiology_events_df.dropna(axis=0, how="any")

display(cleaned_microbiology_events_df.shape)
display(cleaned_microbiology_events_df.head())

(274844, 4)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID
27,106,145167,80023.0,90012.0
28,106,145167,80023.0,90016.0
29,106,145167,80023.0,90006.0
30,106,145167,80023.0,90002.0
31,106,145167,80023.0,90025.0


In [5]:
cleaned_microbiology_events_df.isna().sum()

SUBJECT_ID    0
HADM_ID       0
ORG_ITEMID    0
AB_ITEMID     0
dtype: int64

## Merging with Final Chart Events

In [6]:
FINAL_CHART_EVENTS_PATH = "./data/final_chartevent.csv"

final_chart_events_df = pd.read_csv(FINAL_CHART_EVENTS_PATH, index_col=0) # dropping "Unnamed:0"

display(final_chart_events_df.head())
display(final_chart_events_df.shape)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,VALUE,VALUENUM
0,109,166018,241668.0,220228,8.5,8.5
1,109,166018,241668.0,220545,25.9,25.9
2,109,166018,241668.0,220546,3.3,3.3
3,109,166018,241668.0,220602,110.0,110.0
4,109,166018,241668.0,220615,6.4,6.4


(401692, 6)

In [7]:
final_chart_events_df = final_chart_events_df.rename(columns={"ITEMID": "CHART_ITEMID", "VALUENUM": "CHART_VALUENUM", "VALUE": "CHART_VALUE"})
display(final_chart_events_df.head())

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM
0,109,166018,241668.0,220228,8.5,8.5
1,109,166018,241668.0,220545,25.9,25.9
2,109,166018,241668.0,220546,3.3,3.3
3,109,166018,241668.0,220602,110.0,110.0
4,109,166018,241668.0,220615,6.4,6.4


In [8]:
display(final_chart_events_df.isna().sum())

SUBJECT_ID         0
HADM_ID            0
ICUSTAY_ID        11
CHART_ITEMID       0
CHART_VALUE        0
CHART_VALUENUM     0
dtype: int64

In [9]:
# removing rows with any columns containing NaN values
display(final_chart_events_df.shape)

cleaned_final_chart_events_df = final_chart_events_df.dropna(axis=0, how="any")

display(cleaned_final_chart_events_df.head())
display(cleaned_final_chart_events_df.shape)

(401692, 6)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM
0,109,166018,241668.0,220228,8.5,8.5
1,109,166018,241668.0,220545,25.9,25.9
2,109,166018,241668.0,220546,3.3,3.3
3,109,166018,241668.0,220602,110.0,110.0
4,109,166018,241668.0,220615,6.4,6.4


(401681, 6)

## Keeping only those records which are available in patients_information.csv

In [10]:
PATIENTS_INFO_PATH = "./data/PATIENTS_INFORMATION.csv"

patients_info_df = pd.read_csv(PATIENTS_INFO_PATH, index_col=0)

display(patients_info_df)

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,LOS,AGE,cohort,Obesity,Non.Adherence,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,9973,M,1,100020,282580,1.1011,58,1,0,0,...,0,0,0,0,1,1,0,0,0,0
1,3365,F,0,100103,200434,3.2836,72,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27290,M,1,100137,212691,3.7297,82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9882,M,0,100177,251800,6.5389,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5525,M,1,100473,257484,5.7583,65,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,23549,F,1,199807,212087,35.7563,74,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1527,8772,M,1,199828,215516,4.9366,69,0,0,0,...,1,0,1,0,0,1,0,0,1,0
1528,7029,M,0,199883,247475,1.2379,26,1,0,1,...,0,0,1,0,0,0,0,0,0,0
1529,1931,M,1,199884,265365,2.6764,72,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
patients_info_df.isna().sum()

SUBJECT_ID                                       0
GENDER                                           0
EXPIRE_FLAG                                      0
HADM_ID                                          0
ICUSTAY_ID                                       0
LOS                                              1
AGE                                              0
cohort                                           0
Obesity                                          0
Non.Adherence                                    0
Developmental.Delay.Retardation                  0
Advanced.Heart.Disease                           0
Advanced.Lung.Disease                            0
Schizophrenia.and.other.Psychiatric.Disorders    0
Alcohol.Abuse                                    0
Other.Substance.Abuse                            0
Chronic.Pain.Fibromyalgia                        0
Chronic.Neurological.Dystrophies                 0
Advanced.Cancer                                  0
Depression                     

In [12]:
display(patients_info_df)

cleaned_patients_info_df = patients_info_df.dropna(axis=0, how="any")

display(cleaned_patients_info_df)

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,LOS,AGE,cohort,Obesity,Non.Adherence,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,9973,M,1,100020,282580,1.1011,58,1,0,0,...,0,0,0,0,1,1,0,0,0,0
1,3365,F,0,100103,200434,3.2836,72,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27290,M,1,100137,212691,3.7297,82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9882,M,0,100177,251800,6.5389,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5525,M,1,100473,257484,5.7583,65,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,23549,F,1,199807,212087,35.7563,74,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1527,8772,M,1,199828,215516,4.9366,69,0,0,0,...,1,0,1,0,0,1,0,0,1,0
1528,7029,M,0,199883,247475,1.2379,26,1,0,1,...,0,0,1,0,0,0,0,0,0,0
1529,1931,M,1,199884,265365,2.6764,72,1,0,0,...,0,0,0,0,0,0,0,1,0,0


,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,LOS,AGE,cohort,Obesity,Non.Adherence,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,9973,M,1,100020,282580,1.1011,58,1,0,0,...,0,0,0,0,1,1,0,0,0,0
1,3365,F,0,100103,200434,3.2836,72,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27290,M,1,100137,212691,3.7297,82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9882,M,0,100177,251800,6.5389,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5525,M,1,100473,257484,5.7583,65,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,23549,F,1,199807,212087,35.7563,74,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1527,8772,M,1,199828,215516,4.9366,69,0,0,0,...,1,0,1,0,0,1,0,0,1,0
1528,7029,M,0,199883,247475,1.2379,26,1,0,1,...,0,0,1,0,0,0,0,0,0,0
1529,1931,M,1,199884,265365,2.6764,72,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
cleaned_patients_info_df.isna().sum()

SUBJECT_ID                                       0
GENDER                                           0
EXPIRE_FLAG                                      0
HADM_ID                                          0
ICUSTAY_ID                                       0
LOS                                              0
AGE                                              0
cohort                                           0
Obesity                                          0
Non.Adherence                                    0
Developmental.Delay.Retardation                  0
Advanced.Heart.Disease                           0
Advanced.Lung.Disease                            0
Schizophrenia.and.other.Psychiatric.Disorders    0
Alcohol.Abuse                                    0
Other.Substance.Abuse                            0
Chronic.Pain.Fibromyalgia                        0
Chronic.Neurological.Dystrophies                 0
Advanced.Cancer                                  0
Depression                     

In [14]:
cleaned_final_chart_events_df


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM
0,109,166018,241668.0,220228,8.5,8.5
1,109,166018,241668.0,220545,25.9,25.9
2,109,166018,241668.0,220546,3.3,3.3
3,109,166018,241668.0,220602,110.0,110.0
4,109,166018,241668.0,220615,6.4,6.4
...,...,...,...,...,...,...
401687,10675,195182,250919.0,220050,179.0,179.0
401688,10675,195182,250919.0,220051,87.0,87.0
401689,10675,195182,250919.0,220052,117.0,117.0
401690,10675,195182,250919.0,220210,17.0,17.0


In [15]:
cleaned_final_chart_events_df["ICUSTAY_ID"].astype(np.int64) # for merging

0         241668
1         241668
2         241668
3         241668
4         241668
           ...  
401687    250919
401688    250919
401689    250919
401690    250919
401691    250919
Name: ICUSTAY_ID, Length: 401681, dtype: int64

In [16]:
cleaned_patients_info_df["ICUSTAY_ID"]

0       282580
1       200434
2       212691
3       251800
4       257484
         ...  
1526    212087
1527    215516
1528    247475
1529    265365
1530    211153
Name: ICUSTAY_ID, Length: 1530, dtype: int64

In [17]:
merged_chart_patient_df = pd.merge(cleaned_final_chart_events_df, cleaned_patients_info_df, on=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"], how="inner")

display(merged_chart_patient_df)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,LOS,AGE,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,154,102354,201272.0,220180,62.0,62.0,M,0,2.0859,54,...,0,1,0,0,0,0,0,1,0,0
1,154,102354,201272.0,220181,72.0,72.0,M,0,2.0859,54,...,0,1,0,0,0,0,0,1,0,0
2,154,102354,201272.0,220045,74.0,74.0,M,0,2.0859,54,...,0,1,0,0,0,0,0,1,0,0
3,154,102354,201272.0,220210,23.0,23.0,M,0,2.0859,54,...,0,1,0,0,0,0,0,1,0,0
4,154,102354,201272.0,220179,104.0,104.0,M,0,2.0859,54,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152231,10675,112633,279575.0,225624,32.0,32.0,F,0,2.1027,56,...,0,0,0,0,0,0,0,1,0,0
152232,10675,112633,279575.0,220180,84.0,84.0,F,0,2.1027,56,...,0,0,0,0,0,0,0,1,0,0
152233,10675,112633,279575.0,220181,106.0,106.0,F,0,2.1027,56,...,0,0,0,0,0,0,0,1,0,0
152234,10675,112633,279575.0,220210,30.0,30.0,F,0,2.1027,56,...,0,0,0,0,0,0,0,1,0,0


In [18]:
merged_chart_patient_df.isna().sum()

SUBJECT_ID                                       0
HADM_ID                                          0
ICUSTAY_ID                                       0
CHART_ITEMID                                     0
CHART_VALUE                                      0
CHART_VALUENUM                                   0
GENDER                                           0
EXPIRE_FLAG                                      0
LOS                                              0
AGE                                              0
cohort                                           0
Obesity                                          0
Non.Adherence                                    0
Developmental.Delay.Retardation                  0
Advanced.Heart.Disease                           0
Advanced.Lung.Disease                            0
Schizophrenia.and.other.Psychiatric.Disorders    0
Alcohol.Abuse                                    0
Other.Substance.Abuse                            0
Chronic.Pain.Fibromyalgia      

## Merging with microbiology events

In [19]:
merged_patient_chart_microbic_df = pd.merge(cleaned_microbiology_events_df, merged_chart_patient_df, on=["SUBJECT_ID", "HADM_ID"], how="inner")

display(merged_patient_chart_microbic_df.head())
display(merged_patient_chart_microbic_df.shape)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,422,117029,80023.0,90016.0,299666.0,220180,57.0,57.0,M,1,...,0,0,1,0,0,0,0,1,0,0
1,422,117029,80023.0,90016.0,299666.0,220181,66.0,66.0,M,1,...,0,0,1,0,0,0,0,1,0,0
2,422,117029,80023.0,90016.0,299666.0,220210,22.0,22.0,M,1,...,0,0,1,0,0,0,0,1,0,0
3,422,117029,80023.0,90016.0,299666.0,220277,92.0,92.0,M,1,...,0,0,1,0,0,0,0,1,0,0
4,422,117029,80023.0,90016.0,299666.0,220277,96.0,96.0,M,1,...,0,0,1,0,0,0,0,1,0,0


(2256713, 27)

In [20]:
merged_patient_chart_microbic_df.isna().sum()

SUBJECT_ID                                       0
HADM_ID                                          0
ORG_ITEMID                                       0
AB_ITEMID                                        0
ICUSTAY_ID                                       0
CHART_ITEMID                                     0
CHART_VALUE                                      0
CHART_VALUENUM                                   0
GENDER                                           0
EXPIRE_FLAG                                      0
LOS                                              0
AGE                                              0
cohort                                           0
Obesity                                          0
Non.Adherence                                    0
Developmental.Delay.Retardation                  0
Advanced.Heart.Disease                           0
Advanced.Lung.Disease                            0
Schizophrenia.and.other.Psychiatric.Disorders    0
Alcohol.Abuse                  

In [21]:
merged_patient_chart_microbic_df

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,422,117029,80023.0,90016.0,299666.0,220180,57.0,57.0,M,1,...,0,0,1,0,0,0,0,1,0,0
1,422,117029,80023.0,90016.0,299666.0,220181,66.0,66.0,M,1,...,0,0,1,0,0,0,0,1,0,0
2,422,117029,80023.0,90016.0,299666.0,220210,22.0,22.0,M,1,...,0,0,1,0,0,0,0,1,0,0
3,422,117029,80023.0,90016.0,299666.0,220277,92.0,92.0,M,1,...,0,0,1,0,0,0,0,1,0,0
4,422,117029,80023.0,90016.0,299666.0,220277,96.0,96.0,M,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256708,10675,112633,80002.0,90028.0,279575.0,225624,32.0,32.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256709,10675,112633,80002.0,90028.0,279575.0,220180,84.0,84.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256710,10675,112633,80002.0,90028.0,279575.0,220181,106.0,106.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256711,10675,112633,80002.0,90028.0,279575.0,220210,30.0,30.0,F,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
# merged_patient_chart_microbio_df = merged_patient_chart_microbio_df.drop_duplicates(subset=["ICUSTAY_ID"], keep="first", ignore_index=True)

merged_patient_chart_microbio_df =merged_patient_chart_microbic_df.drop_duplicates(keep="first", ignore_index=True)

In [23]:
display(merged_patient_chart_microbic_df)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,422,117029,80023.0,90016.0,299666.0,220180,57.0,57.0,M,1,...,0,0,1,0,0,0,0,1,0,0
1,422,117029,80023.0,90016.0,299666.0,220181,66.0,66.0,M,1,...,0,0,1,0,0,0,0,1,0,0
2,422,117029,80023.0,90016.0,299666.0,220210,22.0,22.0,M,1,...,0,0,1,0,0,0,0,1,0,0
3,422,117029,80023.0,90016.0,299666.0,220277,92.0,92.0,M,1,...,0,0,1,0,0,0,0,1,0,0
4,422,117029,80023.0,90016.0,299666.0,220277,96.0,96.0,M,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256708,10675,112633,80002.0,90028.0,279575.0,225624,32.0,32.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256709,10675,112633,80002.0,90028.0,279575.0,220180,84.0,84.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256710,10675,112633,80002.0,90028.0,279575.0,220181,106.0,106.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256711,10675,112633,80002.0,90028.0,279575.0,220210,30.0,30.0,F,0,...,0,0,0,0,0,0,0,1,0,0


## Saving the Final Processed File

In [24]:
# Save the processed file

MERGED_PATIENT_CHART_MICROBIO_SAVE_PATH = "./data/processed_patient_final_chart_microbio.csv"

# TODO: Drop any cols?

merged_patient_chart_microbic_df.to_csv(MERGED_PATIENT_CHART_MICROBIO_SAVE_PATH, index=False) # Prevents Unamed:0

# # The whole cell takes: 36.9s
loaded_df = pd.read_csv(MERGED_PATIENT_CHART_MICROBIO_SAVE_PATH)
display(loaded_df)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,422,117029,80023.0,90016.0,299666.0,220180,57.0,57.0,M,1,...,0,0,1,0,0,0,0,1,0,0
1,422,117029,80023.0,90016.0,299666.0,220181,66.0,66.0,M,1,...,0,0,1,0,0,0,0,1,0,0
2,422,117029,80023.0,90016.0,299666.0,220210,22.0,22.0,M,1,...,0,0,1,0,0,0,0,1,0,0
3,422,117029,80023.0,90016.0,299666.0,220277,92.0,92.0,M,1,...,0,0,1,0,0,0,0,1,0,0
4,422,117029,80023.0,90016.0,299666.0,220277,96.0,96.0,M,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256708,10675,112633,80002.0,90028.0,279575.0,225624,32.0,32.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256709,10675,112633,80002.0,90028.0,279575.0,220180,84.0,84.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256710,10675,112633,80002.0,90028.0,279575.0,220181,106.0,106.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256711,10675,112633,80002.0,90028.0,279575.0,220210,30.0,30.0,F,0,...,0,0,0,0,0,0,0,1,0,0


In [25]:
display(merged_patient_chart_microbic_df)

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,GENDER,EXPIRE_FLAG,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,422,117029,80023.0,90016.0,299666.0,220180,57.0,57.0,M,1,...,0,0,1,0,0,0,0,1,0,0
1,422,117029,80023.0,90016.0,299666.0,220181,66.0,66.0,M,1,...,0,0,1,0,0,0,0,1,0,0
2,422,117029,80023.0,90016.0,299666.0,220210,22.0,22.0,M,1,...,0,0,1,0,0,0,0,1,0,0
3,422,117029,80023.0,90016.0,299666.0,220277,92.0,92.0,M,1,...,0,0,1,0,0,0,0,1,0,0
4,422,117029,80023.0,90016.0,299666.0,220277,96.0,96.0,M,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256708,10675,112633,80002.0,90028.0,279575.0,225624,32.0,32.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256709,10675,112633,80002.0,90028.0,279575.0,220180,84.0,84.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256710,10675,112633,80002.0,90028.0,279575.0,220181,106.0,106.0,F,0,...,0,0,0,0,0,0,0,1,0,0
2256711,10675,112633,80002.0,90028.0,279575.0,220210,30.0,30.0,F,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
display(merged_patient_chart_microbic_df.describe())

,SUBJECT_ID,HADM_ID,ORG_ITEMID,AB_ITEMID,ICUSTAY_ID,CHART_ITEMID,CHART_VALUE,CHART_VALUENUM,EXPIRE_FLAG,LOS,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
count,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,...,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06,2.256713e+06
mean,5.973681e+03,1.377462e+05,8.004636e+04,9.001767e+04,2.331154e+05,2.223833e+05,1.057188e+02,1.057188e+02,9.163527e-01,2.007292e+01,...,4.963786e-01,1.862146e-01,4.589418e-02,3.690545e-02,6.243847e-01,6.487391e-01,4.541198e-01,4.160294e-01,1.743771e-01,1.180797e-01
std,3.294815e+03,2.866085e+04,6.522871e+01,7.511511e+00,1.881740e+04,2.483477e+03,4.750459e+02,4.750459e+02,2.768582e-01,1.305535e+01,...,4.999870e-01,3.892799e-01,2.092556e-01,1.885297e-01,4.842815e-01,4.773644e-01,4.978907e-01,4.928986e-01,3.794335e-01,3.227025e-01
min,4.220000e+02,1.023760e+05,8.000200e+04,9.000200e+04,2.004570e+05,2.200450e+05,-6.000000e+02,-6.000000e+02,0.000000e+00,7.421000e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.208000e+03,1.184700e+05,8.001700e+04,9.001200e+04,2.257770e+05,2.201800e+05,1.500000e+01,1.500000e+01,1.000000e+00,6.497500e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.427000e+03,1.184700e+05,8.002600e+04,9.001700e+04,2.257770e+05,2.205460e+05,5.500000e+01,5.500000e+01,1.000000e+00,1.685980e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.492000e+03,1.669340e+05,8.002600e+04,9.002500e+04,2.485690e+05,2.244100e+05,9.700000e+01,9.700000e+01,1.000000e+00,3.400200e+01,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.067500e+04,1.984640e+05,8.022500e+04,9.003100e+04,2.996660e+05,2.278730e+05,1.601100e+05,1.601100e+05,1.000000e+00,3.400200e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [27]:
display(merged_patient_chart_microbic_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2256713 entries, 0 to 2256712
Data columns (total 27 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   SUBJECT_ID                                     int64  
 1   HADM_ID                                        int64  
 2   ORG_ITEMID                                     float64
 3   AB_ITEMID                                      float64
 4   ICUSTAY_ID                                     float64
 5   CHART_ITEMID                                   int64  
 6   CHART_VALUE                                    float64
 7   CHART_VALUENUM                                 float64
 8   GENDER                                         object 
 9   EXPIRE_FLAG                                    int64  
 10  LOS                                            float64
 11  AGE                                            int64  
 12  cohort                                    

None

## Merging with Labevents - # REMEMBER TO DELETE FILE IF RUNNING AGAIN

In [28]:

# IF YOU DON'T WANT TO DELETE - DON'T RUN THIS AGAIN

# # REMEMBER TO DELETE FILE IF RUNNING AGAIN!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# LABEVENTS_PATH = "./data/LABEVENTS.csv"
# CHUNK_SAVE_PATH = "./temp-chunks-lab/chunk_cleaned.csv"

# CHUNK_SIZE = 10e6

# for chunk in pd.read_csv(LABEVENTS_PATH, chunksize=CHUNK_SIZE, iterator=True):
#     print(f"Before: {chunk.shape}")
#     # display(chunk.head())
    
#     cleaned_chunk = chunk.dropna(axis=0, how="any")
    
#     cleaned_chunk.to_csv(CHUNK_SAVE_PATH,
#                 mode="a",      # create or append 
#                 header=True,   # write csv column names - ALL
#                 index=False)
    
#     print(f"After: {cleaned_chunk.shape}")
#     print()
#     # display(cleaned_chunk.head())

In [29]:
LAB_CHUNK_SAVE_PATH = "./temp-chunks-lab/chunk_cleaned.csv"

cleaned_lab_chunk_df = pd.read_csv(LAB_CHUNK_SAVE_PATH)
display(cleaned_lab_chunk_df)

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
1,444,3,145834.0,50902,2101-10-20 16:40:00,99.0,99.0,mEq/L,abnormal
2,447,3,145834.0,50912,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
3,450,3,145834.0,50970,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
4,451,3,145834.0,50971,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal
...,...,...,...,...,...,...,...,...,...
7881766,27428430,96443,103219.0,50862,2109-12-30 01:40:00,2.2,2.2,g/dL,abnormal
7881767,27428431,96443,103219.0,50863,2109-12-30 01:40:00,172.0,172.0,IU/L,abnormal
7881768,27428434,96443,103219.0,50878,2109-12-30 01:40:00,467.0,467.0,IU/L,abnormal
7881769,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.1,mg/dL,abnormal


In [30]:
cleaned_lab_chunk_df = cleaned_lab_chunk_df.rename(columns={"ITEMID": "LAB_ITEMID", "VALUE": "LAB_VALUE", "VALUEUOM": "LAB_VALUEUOM", "FLAG": "LAB_FLAG", "VALUENUM": "LAB_VALUENUM", "CHARTTIME": "LAB_CHARTTIME"})

In [31]:
cleaned_lab_chunk_df.isna().sum()

ROW_ID           0
SUBJECT_ID       0
HADM_ID          0
LAB_ITEMID       0
LAB_CHARTTIME    0
LAB_VALUE        0
LAB_VALUENUM     0
LAB_VALUEUOM     0
LAB_FLAG         0
dtype: int64

In [32]:
cleaned_lab_chunk_df.describe()

,ROW_ID,SUBJECT_ID,HADM_ID,LAB_ITEMID,LAB_CHARTTIME,LAB_VALUE,LAB_VALUENUM,LAB_VALUEUOM,LAB_FLAG
count,7881771,7881771,7881771.0,7881771,7881771,7881771.0,7881771.0,7881771,7881771
unique,7881770,46398,58391.0,493,1436392,19545.0,19354.0,58,3
top,ROW_ID,12613,168201.0,51221,2142-11-08 05:00:00,21.0,21.0,mg/dL,abnormal
freq,2,5565,5485.0,624359,64,59057.0,59057.0,1804294,7845534


In [33]:
cleaned_lab_chunk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7881771 entries, 0 to 7881770
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ROW_ID         object
 1   SUBJECT_ID     object
 2   HADM_ID        object
 3   LAB_ITEMID     object
 4   LAB_CHARTTIME  object
 5   LAB_VALUE      object
 6   LAB_VALUENUM   object
 7   LAB_VALUEUOM   object
 8   LAB_FLAG       object
dtypes: object(9)
memory usage: 541.2+ MB


In [34]:
merged_patient_chart_microbic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2256713 entries, 0 to 2256712
Data columns (total 27 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   SUBJECT_ID                                     int64  
 1   HADM_ID                                        int64  
 2   ORG_ITEMID                                     float64
 3   AB_ITEMID                                      float64
 4   ICUSTAY_ID                                     float64
 5   CHART_ITEMID                                   int64  
 6   CHART_VALUE                                    float64
 7   CHART_VALUENUM                                 float64
 8   GENDER                                         object 
 9   EXPIRE_FLAG                                    int64  
 10  LOS                                            float64
 11  AGE                                            int64  
 12  cohort                                    

In [35]:
cleaned_lab_chunk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7881771 entries, 0 to 7881770
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ROW_ID         object
 1   SUBJECT_ID     object
 2   HADM_ID        object
 3   LAB_ITEMID     object
 4   LAB_CHARTTIME  object
 5   LAB_VALUE      object
 6   LAB_VALUENUM   object
 7   LAB_VALUEUOM   object
 8   LAB_FLAG       object
dtypes: object(9)
memory usage: 541.2+ MB


## Change col type to numeric

In [36]:
cleaned_lab_chunk_df

,ROW_ID,SUBJECT_ID,HADM_ID,LAB_ITEMID,LAB_CHARTTIME,LAB_VALUE,LAB_VALUENUM,LAB_VALUEUOM,LAB_FLAG
0,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
1,444,3,145834.0,50902,2101-10-20 16:40:00,99.0,99.0,mEq/L,abnormal
2,447,3,145834.0,50912,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
3,450,3,145834.0,50970,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
4,451,3,145834.0,50971,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal
...,...,...,...,...,...,...,...,...,...
7881766,27428430,96443,103219.0,50862,2109-12-30 01:40:00,2.2,2.2,g/dL,abnormal
7881767,27428431,96443,103219.0,50863,2109-12-30 01:40:00,172.0,172.0,IU/L,abnormal
7881768,27428434,96443,103219.0,50878,2109-12-30 01:40:00,467.0,467.0,IU/L,abnormal
7881769,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.1,mg/dL,abnormal


In [37]:
# cleaned_lab_chunk_df = pd.to_numeric(cleaned_lab_chunk_df, errors="coerce")

cols = cleaned_lab_chunk_df.columns.drop(["LAB_CHARTTIME", "LAB_VALUEUOM", "LAB_FLAG"])

print(cols)

cleaned_lab_chunk_df[cols] = cleaned_lab_chunk_df[cols].apply(pd.to_numeric, errors='coerce')

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'LAB_ITEMID', 'LAB_VALUE',
       'LAB_VALUENUM'],
      dtype='object')


In [38]:
display(cleaned_lab_chunk_df.isna().sum())

ROW_ID           2
SUBJECT_ID       2
HADM_ID          2
LAB_ITEMID       2
LAB_CHARTTIME    0
LAB_VALUE        2
LAB_VALUENUM     2
LAB_VALUEUOM     0
LAB_FLAG         0
dtype: int64

In [39]:
cleaned_lab_chunk_df = cleaned_lab_chunk_df.dropna(axis=0, how="any")
display(cleaned_lab_chunk_df.isna().sum())

ROW_ID           0
SUBJECT_ID       0
HADM_ID          0
LAB_ITEMID       0
LAB_CHARTTIME    0
LAB_VALUE        0
LAB_VALUENUM     0
LAB_VALUEUOM     0
LAB_FLAG         0
dtype: int64

In [40]:
cleaned_lab_chunk_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7881769 entries, 0 to 7881770
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ROW_ID         float64
 1   SUBJECT_ID     float64
 2   HADM_ID        float64
 3   LAB_ITEMID     float64
 4   LAB_CHARTTIME  object 
 5   LAB_VALUE      float64
 6   LAB_VALUENUM   float64
 7   LAB_VALUEUOM   object 
 8   LAB_FLAG       object 
dtypes: float64(6), object(3)
memory usage: 601.3+ MB


In [41]:
merged_patient_chart_microbic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2256713 entries, 0 to 2256712
Data columns (total 27 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   SUBJECT_ID                                     int64  
 1   HADM_ID                                        int64  
 2   ORG_ITEMID                                     float64
 3   AB_ITEMID                                      float64
 4   ICUSTAY_ID                                     float64
 5   CHART_ITEMID                                   int64  
 6   CHART_VALUE                                    float64
 7   CHART_VALUENUM                                 float64
 8   GENDER                                         object 
 9   EXPIRE_FLAG                                    int64  
 10  LOS                                            float64
 11  AGE                                            int64  
 12  cohort                                    

In [42]:
cleaned_lab_chunk_df["SUBJECT_ID"] = cleaned_lab_chunk_df["SUBJECT_ID"].astype(np.int64)
cleaned_lab_chunk_df["HADM_ID"] = cleaned_lab_chunk_df["HADM_ID"].astype(np.int64)

In [43]:
tqdm.pandas()

In [44]:
cleaned_lab_chunk_df = cleaned_lab_chunk_df.drop_duplicates()


In [50]:
display(cleaned_lab_chunk_df)

,ROW_ID,SUBJECT_ID,HADM_ID,LAB_ITEMID,LAB_CHARTTIME,LAB_VALUE,LAB_VALUENUM,LAB_VALUEUOM,LAB_FLAG
0,443.0,3,145834,50893.0,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
1,444.0,3,145834,50902.0,2101-10-20 16:40:00,99.0,99.0,mEq/L,abnormal
2,447.0,3,145834,50912.0,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
3,450.0,3,145834,50970.0,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
4,451.0,3,145834,50971.0,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal
...,...,...,...,...,...,...,...,...,...
7881766,27428430.0,96443,103219,50862.0,2109-12-30 01:40:00,2.2,2.2,g/dL,abnormal
7881767,27428431.0,96443,103219,50863.0,2109-12-30 01:40:00,172.0,172.0,IU/L,abnormal
7881768,27428434.0,96443,103219,50878.0,2109-12-30 01:40:00,467.0,467.0,IU/L,abnormal
7881769,27428436.0,96443,103219,50885.0,2109-12-30 01:40:00,2.1,2.1,mg/dL,abnormal


In [44]:
merged_all_df = merged_patient_chart_microbic_df.merge(cleaned_lab_chunk_df, on=["SUBJECT_ID", "HADM_ID"], how="inner").progress_apply(lambda x: x)
display(merged_all_df.shape)
display(merged_all_df)

## Ignore following code - for trial ONLY

In [48]:
# LABEVENTS_PATH = "./data/LABEVENTS.csv"
# CHUNK_SAVE_PATH = "./temp-chunks-lab/chunk_cleaned.csv"

# CHUNK_SIZE = 10e6

# for chunk in pd.read_csv(LABEVENTS_PATH, chunksize=CHUNK_SIZE, iterator=True):
#     print(f"Before: {chunk.shape}")
#     # display(chunk.head())
    
#     cleaned_chunk = chunk.dropna(axis=0, how="any")
    
#     cleaned_chunk.to_csv(CHUNK_SAVE_PATH,
#                 mode="a",      # create or append 
#                 header=True,   # write csv column names - ALL
#                 index=False)
    
#     print(f"After: {cleaned_chunk.shape}")
#     print()
#     # display(cleaned_chunk.head())

In [47]:
# LAB_CHUNK_SAVE_PATH = "./temp-chunks-lab/chunk_cleaned.csv"

# cleaned_lab_chunk_df = pd.read_csv(LAB_CHUNK_SAVE_PATH)
# display(cleaned_lab_chunk_df)

In [46]:
# cleaned_lab_chunk_df.isna().sum()

In [45]:
# cleaned_lab_chunk_df.describe()

In [44]:
# cleaned_lab_chunk_df.info()

In [43]:
# cleaned_lab_chunk_df.drop(columns=["CHARTTIME", "VALUE", "VALUEUOM", "FLAG", "ROW_ID"], inplace=True) # if you get a KeyError, it's because of an inplace drop
# display(cleaned_lab_chunk_df.shape)


In [42]:
# display(cleaned_lab_chunk_df.head())
# display(cleaned_lab_chunk_df.info())

In [41]:
# display(merged_chart_microbio_df.head())
# display(merged_chart_microbio_df.info())


In [74]:
# cleaned_lab_chunk_df["SUBJECT_ID"] = pd.to_numeric(cleaned_lab_chunk_df["SUBJECT_ID"], errors="coerce")

In [75]:
# cleaned_lab_chunk_df["HADM_ID"] = pd.to_numeric(cleaned_lab_chunk_df["HADM_ID"], errors="coerce")

In [76]:
# cleaned_lab_chunk_df["ITEMID"] = pd.to_numeric(cleaned_lab_chunk_df["ITEMID"], errors="coerce")
# cleaned_lab_chunk_df["VALUENUM"] = pd.to_numeric(cleaned_lab_chunk_df["VALUENUM"], errors="coerce")

In [40]:
# cleaned_lab_chunk_df.info()

In [39]:
# cleaned_lab_chunk_df.isna().sum()

In [38]:
# cleaned_lab_chunk_df = cleaned_lab_chunk_df.dropna(axis=0, how="any")
# display(cleaned_lab_chunk_df.isna().sum())

In [80]:
# cleaned_lab_chunk_df["SUBJECT_ID"] = cleaned_lab_chunk_df["SUBJECT_ID"].astype(np.int64)
# cleaned_lab_chunk_df["HADM_ID"] = cleaned_lab_chunk_df["HADM_ID"].astype(np.int64)
# cleaned_lab_chunk_df["ITEMID"] = cleaned_lab_chunk_df["ITEMID"].astype(np.int64)

In [37]:
# cleaned_lab_chunk_df.info()

In [36]:
# merged_chart_microbio_df.info()

In [87]:
# merged_all_df = pd.merge(merged_chart_microbio_df, cleaned_lab_chunk_df, on=["SUBJECT_ID", "HADM_ID"], how="inner")
# display(merged_all_df.shape)
# display(merged_all_df)

# MEMORYERROR

In [35]:
# # rename col in chart-microbio
# merged_chart_microbio_df = merged_chart_microbio_df.rename(columns={"ITEMID": "CHART_ITEMID", "VALUENUM": "CHART_VALUENUM"})
# display(merged_chart_microbio_df.head())

In [34]:
# cleaned_lab_chunk_df = cleaned_lab_chunk_df.rename(columns={"ITEMID": "LAB_ITEMID", "VALUENUM": "LAB_VALUENUM"})
# display(cleaned_lab_chunk_df.head())

In [33]:
# display(merged_chart_microbio_df.shape)
# display(cleaned_lab_chunk_df.shape)

In [32]:
# cleaned_lab_chunk_df.isna().sum()

In [31]:
# merged_chart_microbio_df = merged_chart_microbio_df.drop_duplicates(ignore_index=True) # keeps first by default
# display(merged_chart_microbio_df.shape)

In [30]:
# cleaned_lab_chunk_df = cleaned_lab_chunk_df.drop_duplicates(ignore_index=True)
# display(cleaned_lab_chunk_df.shape)

In [29]:
# merged_all_df = pd.merge(merged_chart_microbio_df, cleaned_lab_chunk_df, on=["SUBJECT_ID", "HADM_ID"], how="inner")
# display(merged_all_df.shape)
# display(merged_all_df)

In [28]:
# cleaned_lab_chunk_df = pd.read_csv(CLEANED_LAB_CHUNK_SAVE_PATH, index_col=False)

# # creating a empty bucket to save result
# MERGE_3_CHUNK_SAVE_PATH = "./data/merged_chart_microbio_lab.csv"

# df_result = pd.DataFrame(columns=(merged_chart_microbio_df.columns.append(cleaned_lab_chunk_df.columns)).unique())
# df_result.to_csv(MERGE_3_CHUNK_SAVE_PATH, index_label=False)

# # save data which only appear in df1 # sorry I was doing left join here. no need to run below two line.
# # df_result = df1[df1.Colname1.isin(df2.Colname2)!=True]
# # df_result.to_csv("df3.csv",index_label=False, mode="a")

# CLEANED_LAB_CHUNK_SAVE_PATH = "./data/cleaned_lab_chunk_before_merge.csv"
# cleaned_lab_chunk_df.to_csv(CLEANED_LAB_CHUNK_SAVE_PATH, index=False)

# # deleting df2 to save memory
# # del(cleaned_lab_chunk_df)

# # merged_all_df = pd.merge(merged_chart_microbio_df, cleaned_lab_chunk_df, on=["SUBJECT_ID", "HADM_ID"], how="inner")
# # display(merged_all_df.shape)
# # display(merged_all_df)

# def preprocess(x):
#     df2=pd.merge(merged_chart_microbio_df, x, on=["SUBJECT_ID", "HADM_ID"], how="inner")
#     df2.to_csv(MERGE_3_CHUNK_SAVE_PATH, mode="a", header=True, index=False)

# reader = pd.read_csv(CLEANED_LAB_CHUNK_SAVE_PATH, chunksize=10e6) # chunksize depends with you colsize

# [preprocess(r) for r in reader]